(working-with-data)=
# Working with Data


## Introduction

The previous chapter was just a quick tour of what can be done with a single tabular dataset (a 'dataframe'). In this chapter, we'll go deeper into working with data.

The ability to extract, clean, and analyse data is one of the core skills any economist needs. Fortunately, the (open source) tools that are available for data analysis have improved enormously in recent years, and working with them can be a delight--even the most badly formatted data can be beaten into shape.

In this chapter, you'll get really good introduction to the [**pandas**](https://pandas.pydata.org/) package, the core data manipulation library in Python. The name is derived from 'panel data' but it's suited to any tabular data, and can be used to work with more complex data structures too. We *won't* cover reading in or writing data here; see the next chapter for that.

This chapter is hugely indebted to the fantastic [Python Data Science Handbook](https://jakevdp.github.io/PythonDataScienceHandbook/) by Jake Vanderplas. Remember, if you get stuck with pandas, there is brilliant [documentation](https://pandas.pydata.org/docs/user_guide/index.html) and a fantastic set of [introductory tutorials](https://pandas.pydata.org/pandas-docs/stable/getting_started/intro_tutorials/index.html) on their website. These notes are heavily indebted to those introductory tutorials.

This chapter uses the **pandas**, **seaborn**, and **numpy** packages. If you're running this code, you may need to install these packages, which you can do using either `conda install packagename` or `pip install packagename` on your computer's command line. (If you're not sure what a command line or terminal is, take a quick look at the basics of coding chapter.)


### Using tidy data

As an aside, if you're working with tabular data, it's good to try and use a so-called 'tidy data' format. This is data with one observation per row, and one variable per column, like so:

![](https://d33wubrfki0l68.cloudfront.net/6f1ddb544fc5c69a2478e444ab8112fb0eea23f8/91adc/images/tidy-1.png)

Tidy data aren't going to be appropriate *every* time and in every case, but they're a really, really good default for tabular data. Once you use it as your default, it's easier to think about how to perform subsequent operations. Some plotting libraries, such as **seaborn**, take that your data are in tidy format as a given. And many operations that you can perform on dataframes (the objects that hold tabular data within many programming languages) are easier when you have tidy data. If you're writing out data to file to share, putting it in tidy format is a really good idea.

Of course, *getting* your messy dataset into a tidy format may take a bit of work... but we're about to enter the exciting world of coding for data analysis and the tools you'll see in the rest of this chapter will help you to 'wrangle' even the most messy of datasets.

Having said that tidy data are great, and they are, one of standard data library **pandas**' advantages relative to other data analysis libraries is that it isn't *too* tied to tidy data and can navigate awkward non-tidy data manipulation tasks happily too.

## Dataframes and series

Let's start with the absolute basics. The most basic **pandas** object is a dataframe. A DataFrame is a 2-dimensional data structure that can store data of different types (including characters, integers, floating point values, categorical data, even lists) in columns. 

![](https://pandas.pydata.org/docs/_images/01_table_dataframe.svg)

We'll now import some useful packages, set a random number seed (some examples use randomly generated data), set up some nice plot settings, and load a dataset.

We'll look at a dataframe of the *penguins* dataset. To show just the first 5 rows, I'll be using the `head()` method (there's also a `tail()` method).


In [1]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Set seed for reproducibility
np.random.seed(10)
# Plot settings
plt.style.use(
    "https://github.com/aeturrell/coding-for-economists/raw/main/plot_style.txt"
)

df = sns.load_dataset("penguins")
df.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female


What just happened? We loaded a pandas dataframe called `df` and showed its contents. You can see the column names in bold, and the index on the left hand side. Just to double check it *is* a pandas dataframe, we can call type on this.

In [2]:
type(df)

pandas.core.frame.DataFrame

And if we want a bit more information about what we imported (including the datatypes of the columns):

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            344 non-null    object 
 1   island             344 non-null    object 
 2   bill_length_mm     342 non-null    float64
 3   bill_depth_mm      342 non-null    float64
 4   flipper_length_mm  342 non-null    float64
 5   body_mass_g        342 non-null    float64
 6   sex                333 non-null    object 
dtypes: float64(4), object(3)
memory usage: 18.9+ KB


Remember that everything in Python is an object, and our dataframe is no exception. Each dataframe is made up of a set of series that, in a dataframe, become columns: but you can turn a single series into a dataframe too. 

![](https://pandas.pydata.org/docs/_images/01_table_series.svg)

Let's see a couple of ways of creating some series from raw data:

In [4]:
# From a list:
s1 = pd.Series([1.0, 6.0, 19.0, 2.0])
print(s1)
print("\n")
# From a dictionary
population_dict = {
    "California": 38332521,
    "Texas": 26448193,
    "New York": 19651127,
    "Florida": 19552860,
    "Illinois": 12882135,
}
s2 = pd.Series(population_dict)
print(s2)

0     1.0
1     6.0
2    19.0
3     2.0
dtype: float64


California    38332521
Texas         26448193
New York      19651127
Florida       19552860
Illinois      12882135
dtype: int64


Note that in each case there is no column name (because this is a series, not a dataframe), and there *is* an index. The index is automatically created if we don't specify it; in the third example, by passing a dictionary we implicitly asked for the index to be the locations we supplied. 

If you ever need to get the data 'out' of a series or dataframe, you can just call the `values` method on the object:

In [5]:
s2.values

array([38332521, 26448193, 19651127, 19552860, 12882135])

If you ever want to turn a series into a dataframe, just called `pd.DataFrame(series)` on it. Note that while series have an index and an object name (eg `s2` above), they don't have any column labels because they only have one column.

Now let's try creating our own dataframe with more than one column of data using a *dictionary*:

In [6]:
df = pd.DataFrame(
    {
        "A": 1.0,
        "B": pd.Series(1, index=list(range(4)), dtype="float32"),
        "C": [3] * 4,
        "D": pd.Categorical(["test", "train", "test", "train"]),
        "E": "foo",
    }
)
df

,A,B,C,D,E
0,1.0,1.0,3,test,foo
1,1.0,1.0,3,train,foo
2,1.0,1.0,3,test,foo
3,1.0,1.0,3,train,foo


Remember, curly brackets in the format `{key: value}` denote a dictionary. In the above example, the `pd.DataFrame()` function understands that any single value entries in the dictionary that is passed, such as `{'A': 1.}`, should be repeated as many times as are needed to match the longest series in the dictionary (4 in the above example).

Another way to create dataframes is to pass a bunch of series (note that `index`, `columns`, and `dtype` are optional--you can just specify the data):

In [7]:
df = pd.DataFrame(
    data=np.reshape(range(36), (6, 6)),
    index=["a", "b", "c", "d", "e", "f"],
    columns=["col" + str(i) for i in range(6)],
    dtype=float,
)
df

,col0,col1,col2,col3,col4,col5
a,0.0,1.0,2.0,3.0,4.0,5.0
b,6.0,7.0,8.0,9.0,10.0,11.0
c,12.0,13.0,14.0,15.0,16.0,17.0
d,18.0,19.0,20.0,21.0,22.0,23.0
e,24.0,25.0,26.0,27.0,28.0,29.0
f,30.0,31.0,32.0,33.0,34.0,35.0


Note that `reshape` takes an input and puts it into a given shape (a 6 by 6 matrix in the above example).

### Values, columns, and index

You'll have seen that there are three different things that make up a dataframe: the values that are in the cells, the column names, and the index. The column and index can take on values that are the same as the values in a dataframe do; string, int, float, datetime, and more. It's pretty obvious what role the columns play: they keep track of the name of different sets of values. But for people who may have seen other dataframe-like libraries, the role played by the index may be less familiar. The easiest way to think about a **pandas** index is that it does for row values what the columns do for columnar values: it's a way of keeping track of what individual roles are and it *doesn't* get used for calculations (just as summing a column ignores the name of the row).

Here's an example to show this. Let's first create a simple dataframe:

In [8]:
df = pd.DataFrame(
    data={
        "col0": [0, 0, 0, 0],
        "col1": [0, 0, 0, 0],
        "col2": [0, 0, 0, 0],
        "col3": ["a", "b", "b", "a"],
        "col4": ["alpha", "gamma", "gamma", "gamma"],
    },
    index=["row" + str(i) for i in range(4)],
)
df.head()

,col0,col1,col2,col3,col4
row0,0,0,0,a,alpha
row1,0,0,0,b,gamma
row2,0,0,0,b,gamma
row3,0,0,0,a,gamma


If we add one to the integer columns in the dataframe, this is what we get (note we're not saving the result):

In [9]:
df[["col0", "col1", "col2"]] + 1

,col0,col1,col2
row0,1,1,1
row1,1,1,1
row2,1,1,1
row3,1,1,1


Now let's use `col0` as our index instead of the original labels we created and add one to the remaining numeric columns:

In [10]:
df = df.set_index("col0")
df[["col1", "col2"]] = df[["col1", "col2"]] + 1
df.head()

,col1,col2,col3,col4
col0,,,,
0,1,1,a,alpha
0,1,1,b,gamma
0,1,1,b,gamma
0,1,1,a,gamma


What was a column name has become an index name (which you can change with `df.index.name='newname'`) and, when we do add one, it isn't applied to the index. Even though their datatype is `int`, for integer, the index entries are now acting as a label for each row--not as values in the dataframe.

An index can be useful for keeping track of what's going on, and it's particularly convenient for some datetime operations.

Whenever you use `groupby` (and some other operations), the columns you use to perform the operation are set as the index of the returned dataframe (you can have multiple index columns). To get back those back to being columns, use the `reset_index()` method like so:

In [11]:
df.groupby(["col3", "col4"]).sum()

col1  col2
col3 col4             
a    alpha     1     1
     gamma     1     1
b    gamma     2     2

In [12]:
df.groupby(["col3", "col4"]).sum().reset_index()

,col3,col4,col1,col2
0,a,alpha,1,1
1,a,gamma,1,1
2,b,gamma,2,2


## Datatypes

Pandas has some built-in datatypes (some are the basic Python datatypes) that will make your life a *lot* easier if you work with them. Why bother specifying datatypes? Languages like Python let you get away with having pretty much anything in your columns. But this can be a problem: sometimes you'll end up mixing integers, strings, the generic 'object' datatype, and more by mistake. By ensuring that columns conform to a datatype, you can save yourself from some of the trials that come with these mixed datatypes. Some of the most important datatypes for dataframe are string, float, categorical, datetime, int, and boolean. 

Typically, you'll read in a dataset where the dataypes of the columns are a mess. One of the first things you'll want to do is sort these out. Here's an example dataset showing how to set the datatypes:

In [13]:
data = [
    ["string1", "string2"],
    [1.2, 3.4],
    ["type_a", "type_b"],
    ["01-01-1999", "01-01-2000"],
    [1, 2],
    [0, 1],
]
columns = [
    "string_col",
    "double_col",
    "category_col",
    "datetime_col",
    "integer_col",
    "bool_col",
]

df = pd.DataFrame(data=np.array(data).T, columns=columns)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   string_col    2 non-null      object
 1   double_col    2 non-null      object
 2   category_col  2 non-null      object
 3   datetime_col  2 non-null      object
 4   integer_col   2 non-null      object
 5   bool_col      2 non-null      object
dtypes: object(6)
memory usage: 224.0+ bytes


Note that the data type for all of these columns is the generic 'Object' (you can see this from the `Dtype` column that is printed when you use `df.info()`). Let's fix that:


In [14]:
df = df.assign(
    string_col=df["string_col"].astype("string"),
    double_col=df["double_col"].astype("double"),
    category_col=df["category_col"].astype("category"),
    datetime_col=df["datetime_col"].astype("datetime64[ns]"),
    integer_col=df["integer_col"].astype("int"),
    bool_col=df["bool_col"].astype("bool"),
)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   string_col    2 non-null      string        
 1   double_col    2 non-null      float64       
 2   category_col  2 non-null      category      
 3   datetime_col  2 non-null      datetime64[ns]
 4   integer_col   2 non-null      int64         
 5   bool_col      2 non-null      bool          
dtypes: bool(1), category(1), datetime64[ns](1), float64(1), int64(1), string(1)
memory usage: 320.0 bytes


Once you have applied datatypes to the columns in your dataframe, tools like [skimpy]() (installed using `pip install skimpy` can then provide richer summaries of your data. 

In [15]:
from skimpy import skim

skim(df)

╭───────────────────────────────────── skimpy summary ──────────────────────────────────────╮
│          Data Summary                Data Types               Categories                  │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓ ┏━━━━━━━━━━━━━━━━━━━━━━━┓          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃ ┃ Categorical Variables ┃          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩ ┡━━━━━━━━━━━━━━━━━━━━━━━┩          │
│ │ Number of rows    │ 2      │ │ string      │ 1     │ │ category_col          │          │
│ │ Number of columns │ 6      │ │ float64     │ 1     │ └───────────────────────┘          │
│ └───────────────────┴────────┘ │ category    │ 1     │                                    │
│                                │ datetime64  │ 1     │                                    │
│                                │ int64       │ 1     │                                    │
│                                │ bool        │ 1     │                                    │
│                                └─────────────┴───────┘                                    │
│                                          number                                           │
│ ┏━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━┳━━━━━┳━━━━━┳━━━━━┳━━━━━━┳━━━━━━━━┓  │
│ ┃             ┃ missing ┃ complete     ┃ mean ┃ sd   ┃ p0  ┃ p25 ┃ p75 ┃ p100 ┃ hist   ┃  │
│ ┃             ┃         ┃ rate         ┃      ┃      ┃     ┃     ┃     ┃      ┃        ┃  │
│ ┡━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━╇━━━━━╇━━━━━╇━━━━━╇━━━━━━╇━━━━━━━━┩  │
│ │ double_col  │ 0.0     │ 1.0          │ 2.3  │ 1.6  │ 1.2 │ 1.8 │ 2.8 │ 3.4  │ █    █ │  │
│ │ integer_col │ 0.0     │ 1.0          │ 1.5  │ 0.71 │ 1.0 │ 1.2 │ 1.8 │ 2.0  │ █    █ │  │
│ └─────────────┴─────────┴──────────────┴──────┴──────┴─────┴─────┴─────┴──────┴────────┘  │
│                                         category                                          │
│ ┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━┓  │
│ ┃                     ┃ missing      ┃ complete rate         ┃ ordered     ┃ unique    ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━┩  │
│ │ category_col        │ 0.0          │ 1.0                   │ False       │ 2.0       │  │
│ └─────────────────────┴──────────────┴───────────────────────┴─────────────┴───────────┘  │
│                                         datetime                                          │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓  │
│ ┃                   ┃ missing    ┃ complete rate      ┃ first          ┃ last          ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩  │
│ │ datetime_col      │ 0.0        │ 1.0                │ 1999-01-01     │ 2000-01-01    │  │
│ └───────────────────┴────────────┴────────────────────┴────────────────┴───────────────┘  │
│                                          string                                           │
│ ┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┓  │
│ ┃                ┃ missing    ┃ complete rate     ┃ words per row     ┃ total words    ┃  │
│ ┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━┩  │
│ │ string_col     │ 0.0        │ 1.0               │ 1.0               │ 2.0            │  │
│ └────────────────┴────────────┴───────────────────┴───────────────────┴────────────────┘  │
│                                           bool                                            │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓  │
│ ┃                        ┃ true          ┃ true rate                ┃ hist             ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩  │
│ │ bool_col               │ 2.0           │ 1.0    

There's a lot more on exploratory data analysis in the Chapter on {ref}`data-exploratory-analysis`.

If you're creating a series or dataframe from scratch, here's how to start off with these datatypes:

In [16]:
import pandas as pd
import numpy as np
str_s = pd.Series(["string1", "string2"], dtype="string")
float_s = pd.Series([1.2, 3.4], dtype=float)
cat_s = pd.Series(["type_a", "type_b"], dtype="category")
date_s = pd.Series(["01-01-1999", "01-01-2000"], dtype="datetime64[ns]")
int_s = pd.Series([1, 2], dtype="int")
bool_s = pd.Series([True, False], dtype=bool)

df = pd.concat([str_s, float_s, cat_s, date_s, int_s, bool_s], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   0       2 non-null      string        
 1   1       2 non-null      float64       
 2   2       2 non-null      category      
 3   3       2 non-null      datetime64[ns]
 4   4       2 non-null      int64         
 5   5       2 non-null      bool          
dtypes: bool(1), category(1), datetime64[ns](1), float64(1), int64(1), string(1)
memory usage: 320.0 bytes


### Categorical variables

Categorical variables can be especially useful. They imply a finite set of possible values, for example, different UK counties. They have datatype `"category"` as shown above but can be created from elements that have whatever datatype you like (eg, also above, the "type_a" and "type_b" values have datatype string before being cast as "category"). In the context of regression, you might call these "fixed effects" variables.

As well as using `.astype("category")` on a column to convert it directly to be of categorical type, there are a couple of convenience functions that allow you to create them from other types of columns. `pd.cut` splits input data into a given number of (evenly spaced) bins that you can (optionally) give names to via the `labels=` keyword. The default behaviour is for the order of the labels to matter so that, in the below example, the bins will be evenly spaced from the smallest value to the largest, with the smallest receiving the label `bad`.

In [17]:
pd.cut([1, 7, 5, 4, 6, 3], 3, labels=["bad", "medium", "good"])

['bad', 'good', 'medium', 'medium', 'good', 'bad']
Categories (3, object): ['bad' < 'medium' < 'good']

We can also pass the bins directly:

In [18]:
pd.cut([1, 7, 5, 4, 6, 3], bins=[-5, 0, 5, 10])

[(0, 5], (5, 10], (0, 5], (0, 5], (5, 10], (0, 5]]
Categories (3, interval[int64, right]): [(-5, 0] < (0, 5] < (5, 10]]

Another useful function is qcut, which provides a categorical breakdown according to a given number of quantiles (eg 4 produces quartiles):

In [19]:
pd.qcut(range(1, 10), 4)

[(0.999, 3.0], (0.999, 3.0], (0.999, 3.0], (3.0, 5.0], (3.0, 5.0], (5.0, 7.0], (5.0, 7.0], (7.0, 9.0], (7.0, 9.0]]
Categories (4, interval[float64, right]): [(0.999, 3.0] < (3.0, 5.0] < (5.0, 7.0] < (7.0, 9.0]]

## Accessing and slicing

Now you know how to put data in a dataframe, how do you access the bits of it you need? There are various ways. If you want to access an entire column, the syntax is very simple; `df['columname']` (you can also use `df.columname`).

![](https://pandas.pydata.org/docs/_images/03_subset_columns.svg)




In [20]:
df = pd.DataFrame(
    data=np.reshape(range(36), (6, 6)),
    index=["a", "b", "c", "d", "e", "f"],
    columns=["col" + str(i) for i in range(6)],
    dtype=float,
)
df["col1"]

a     1.0
b     7.0
c    13.0
d    19.0
e    25.0
f    31.0
Name: col1, dtype: float64

To access a particular row, it's `df.loc['rowname']` or `df.loc[['rowname1', 'rowname1']]`.

![](https://pandas.pydata.org/docs/_images/03_subset_rows.svg)


In [21]:
df.loc[["a", "b"]]

,col0,col1,col2,col3,col4,col5
a,0.0,1.0,2.0,3.0,4.0,5.0
b,6.0,7.0,8.0,9.0,10.0,11.0


As well as the `.loc` method, there is the `.iloc` method that accesses rows or columns based on their position, for example `df.iloc[i, :]` for the ith row and `df.iloc[:, j]` for the jth column (but remember the numbers start from zero).

To access an individual value from within the dataframe, we have two options: pass an index value and a column name to `.loc[rowname, columnname]` or retrieve the value by using its position using `.iloc[row, column]`: 

In [22]:
# Using .loc
print(df.loc["b", "col1"])
# Using .iloc
print(df.iloc[1, 0])

7.0
6.0


With all of these different ways to access values in dataframes, it can get confusing. These are the different ways to get the first column of a dataframe (when that first column is called `column` and the dataframe is `df`):

- `df.column`
- `df["column"]`
- `df.loc[:, "column"]`
- `df.iloc[:, 0]`

The ways to access rows are similar (here assuming the first row is called `row`):

- `df.loc["row", :]`
- `df.iloc[0, :]`

And to access the first value (ie the value in first row, first column):

- `df.column[0]`
- `df["column"][0]`
- `df.iloc[0, 0]`
- `df.loc["row", "column"]`

In the above examples, square brackets are instructions about *where* to grab bits from the dataframe. They are a bit like an address system for values within a dataframe.

Square brackets *also* denote lists though. So if you want to select *multiple* columns or rows, you might see syntax like this:

`df.loc[["row0", "row1"], ["column0", "column2"]]`

which picks out two rows and two columns via the lists `["row0", "row1"]` and `["column0", "column2"]`. Because there are lists alongside the usual system of selecting values, there are two sets of square brackets.

So often what we really want is a subset of values (as opposed to *all* values or just *one* value). This is where *slicing* comes in. If you've looked at the Basics of Coding chapter, you'll know a bit about slicing and indexing already, but we'll cover the basics here too.

The syntax for slicing is similar to what we've seen already: there are two methods `.loc` to access items by name, and `.iloc` to access them by position. The syntax for the former is `df.loc[start:stop:step, start:stop:step]`, where the first position is index name and the second is column name (and the same applies for numbers and `df.iloc`). Let's see an example using the storms dataset, and do some cuts.

In [23]:
df.loc["a":"f":2, "col1":"col3"]

,col1,col2,col3
a,1.0,2.0,3.0
c,13.0,14.0,15.0
e,25.0,26.0,27.0


As you can see, slicing even works on names! By asking for rows `'a':'f':2`, we get every other row from 'a' to 'f' (inclusive). Likewise, for columns, we asked for every column between `col1` and `col3` (inclusive). `iloc` works in a very similar way.

In [24]:
df.iloc[1:, :-1]

,col0,col1,col2,col3,col4
b,6.0,7.0,8.0,9.0,10.0
c,12.0,13.0,14.0,15.0,16.0
d,18.0,19.0,20.0,21.0,22.0
e,24.0,25.0,26.0,27.0,28.0
f,30.0,31.0,32.0,33.0,34.0


In this case, we asked for everything from row 1 onwards, and everything up to (but excluding) the last column.

It's not just strings and positions that can be sliced though, here's an example using *dates* (pandas support for dates is truly excellent):

In [25]:
index = pd.date_range("1/1/2000", periods=12, freq="Q")
df = pd.DataFrame(np.random.randint(0, 10, (12, 5)), index=index, columns=list("ABCDE"))
df

,A,B,C,D,E
2000-03-31,9,4,0,1,9
2000-06-30,0,1,8,9,0
2000-09-30,8,6,4,3,0
2000-12-31,4,6,8,1,8
2001-03-31,4,1,3,6,5
2001-06-30,3,9,6,9,1
2001-09-30,9,4,2,6,7
2001-12-31,8,8,9,2,0
2002-03-31,6,7,8,1,7
2002-06-30,1,4,0,8,5


Now let's do some slicing!

In [26]:
df.loc["2000-01-01":"2002-01-01", :]

,A,B,C,D,E
2000-03-31,9,4,0,1,9
2000-06-30,0,1,8,9,0
2000-09-30,8,6,4,3,0
2000-12-31,4,6,8,1,8
2001-03-31,4,1,3,6,5
2001-06-30,3,9,6,9,1
2001-09-30,9,4,2,6,7
2001-12-31,8,8,9,2,0


Two important points to note here: first, pandas doesn't mind that we supplied a date that didn't actually exist in the index. It worked out that by '2000-01-01' we meant a datetime and compared the values of the index to that datetime in order to decide what rows to return from the dataframe. The second thing to notice is the use of `:` for the column names; this explicitly says 'give me all the columns'.

## Operations on dataframes

Columns in dataframes can undergo all the usual arithmetic operations you'd expect of addition, multiplication, division, and so on. If the underlying datatypes of two columns have a group operation, then the dataframe columns will use that. 

![](https://github.com/pandas-dev/pandas/raw/059c8bac51e47d6eaaa3e36d6a293a22312925e6/doc/source/_static/schemas/05_newcolumn_1.svg)

The results of these manipulations can just be saved as a new series, eg, `new_series = df['A'] + df['B']` or created as a new column of the dataframe:

In [27]:
df["new_col"] = df["A"] * (df["B"] ** 2) + 1
df

,A,B,C,D,E,new_col
2000-03-31,9,4,0,1,9,145
2000-06-30,0,1,8,9,0,1
2000-09-30,8,6,4,3,0,289
2000-12-31,4,6,8,1,8,145
2001-03-31,4,1,3,6,5,5
2001-06-30,3,9,6,9,1,244
2001-09-30,9,4,2,6,7,145
2001-12-31,8,8,9,2,0,513
2002-03-31,6,7,8,1,7,295
2002-06-30,1,4,0,8,5,17


Boolean variables and strings have group operations (eg concatenation is via `+` with strings), and so work well with column operations too:

In [28]:
df = pd.DataFrame(
    {"a": [1, 0, 1], "b": [0, 1, 1], "c": [0, 1, 1], "d": [1, 1, 0]}, dtype=bool
)
print(df)
print("\n a and c:\n")
print(df["a"] & df["c"])
print("\n b or d:\n")
print(df["b"] | df["d"])

       a      b      c      d
0   True  False  False   True
1  False   True   True   True
2   True   True   True  False

 a and c:

0    False
1    False
2     True
dtype: bool

 b or d:

0    True
1    True
2    True
dtype: bool


More complex operations on whole dataframes are supported, but if you're doing very heavy lifting you might want to just switch to using numpy arrays (**numpy** is basically Matlab in Python). As examples though, you can transpose and exponentiate easily:

In [29]:
df = pd.DataFrame(np.random.randint(0, 5, (3, 5)), columns=list("ABCDE"))
print("\n Dataframe:")
print(df)
print("\n Exponentiation:")
print(np.exp(df))
print("\n Transpose:")
print(df.T)


 Dataframe:
   A  B  C  D  E
0  2  3  0  4  4
1  0  1  1  4  0
2  2  1  3  1  2

 Exponentiation:
          A          B          C          D          E
0  7.389056  20.085537   1.000000  54.598150  54.598150
1  1.000000   2.718282   2.718282  54.598150   1.000000
2  7.389056   2.718282  20.085537   2.718282   7.389056

 Transpose:
   0  1  2
A  2  0  2
B  3  1  1
C  0  1  3
D  4  4  1
E  4  0  2


## Aggregation

**pandas** has built-in aggregation functions such as

| Aggregation      | Description |
| ----------- | ----------- |
| `count()`      | Number of items       |
| `first()`, `last()` | 	First and last item |
| `mean()`, `median()` |	Mean and median |
| `min()`, `max()` |	Minimum and maximum |
| `std()`, `var()` |	Standard deviation and variance |
| `mad()` |	Mean absolute deviation |
| `prod()` |	Product of all items |
| `sum()`	| Sum of all items |
| `value_counts()` | Counts of unique values |

these can applied to all entries in a dataframe, or optionally to rows or columns using `axis=0` or `axis=1` respectively.


In [30]:
df.sum(axis=0)

A    4
B    5
C    4
D    9
E    6
dtype: int64

## Split, apply, and combine

Splitting a dataset, applying a function, and combining the results are three key operations that we'll want to use again and again. Splitting means differentiating between rows or columns of data based on some conditions, for instance different categories or different values. Applying means applying a function, for example finding the mean or sum. Combine means putting the results of these operations back into the dataframe, or into a variable. The figure gives an example

![](https://jakevdp.github.io/PythonDataScienceHandbook/figures/03.08-split-apply-combine.png)

Note that the 'combine' part doesn't always have to result in a new dataframe; it could create new columns in an existing dataframe.

Let's first see a really simple example of splitting a dataset into groups and finding the mean across those groups using the *penguins* dataset. We'll group the data by island and look at the means. 

In [31]:
df = sns.load_dataset("penguins")
df.groupby("island").mean()

,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g
island,,,,
Biscoe,45.257485,15.874850,209.706587,4716.017964
Dream,44.167742,18.344355,193.072581,3712.903226
Torgersen,38.950980,18.429412,191.196078,3706.372549


The aggregations from the previous part all work on grouped data. An example is `df['body_mass_g'].groupby('island').std()` for the standard deviation of body mass by island.

You can also pass other functions via the `agg` method (short for aggregation). Here we pass two numpy functions:


In [32]:
df.groupby("species").agg([np.mean, np.std])

bill_length_mm           bill_depth_mm           flipper_length_mm  \
                    mean       std          mean       std              mean   
species                                                                        
Adelie         38.791391  2.663405     18.346358  1.216650        189.953642   
Chinstrap      48.833824  3.339256     18.420588  1.135395        195.823529   
Gentoo         47.504878  3.081857     14.982114  0.981220        217.186992   

                     body_mass_g              
                std         mean         std  
species                                       
Adelie     6.539457  3700.662252  458.566126  
Chinstrap  7.131894  3733.088235  384.335081  
Gentoo     6.484976  5076.016260  504.116237

Multiple aggregations can also be performed at once on the entire dataframe by using a dictionary to map columns into functions. You can also group by as many variables as you like by passing the groupby method a list of variables. Here's an example that combines both of these features:


In [33]:
df.groupby(["species", "island"]).agg({"body_mass_g": "sum", "bill_length_mm": "mean"})

body_mass_g  bill_length_mm
species   island                                
Adelie    Biscoe        163225.0       38.975000
          Dream         206550.0       38.501786
          Torgersen     189025.0       38.950980
Chinstrap Dream         253850.0       48.833824
Gentoo    Biscoe        624350.0       47.504878

Sometimes, inheriting the column names becomes problematic. There's a slightly fussy syntax to help with that:


In [34]:
df.groupby(["species", "island"]).agg(
    count_bill=("bill_length_mm", "count"),
    mean_bill=("bill_length_mm", "mean"),
    std_flipper=("flipper_length_mm", np.std),
)

count_bill  mean_bill  std_flipper
species   island                                       
Adelie    Biscoe             44  38.975000     6.729247
          Dream              56  38.501786     6.585083
          Torgersen          51  38.950980     6.232238
Chinstrap Dream              68  48.833824     7.131894
Gentoo    Biscoe            123  47.504878     6.484976

Finally, you should know about the `apply` method, which takes a function and applies it to a given axis (`axis=0` for index, `axis=1` for columns) or column. The simple example below shows how it works, though in practice you'd just use `df['body_mass_kg'] = df['body_mass_g]/1e3` to do this.

In [35]:
def g_to_kg(mass_in_g):
    return mass_in_g / 1e3


df["mass_in_kg"] = df["body_mass_g"].apply(g_to_kg)
df.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,mass_in_kg
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male,3.75
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female,3.80
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female,3.25
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female,3.45


## Filter, transform, apply, and assign

### Filter

Filtering does exactly what it sounds like, but it can make use of group-by commands. In the example below, all but one species is filtered out.

In the example below, `filter` passes a grouped version of the dataframe into the `filter_func` we've defined (imagine that a dataframe is passed for each group). Because the passed variable is a dataframe, and variable `x` is defined in the function, the `x` within `filter_func` body behaves like our dataframe--including having the same columns.

In [36]:
def filter_func(x):
    return x["bill_length_mm"].mean() > 48


df.groupby("species").filter(filter_func).head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,mass_in_kg
152,Chinstrap,Dream,46.5,17.9,192.0,3500.0,Female,3.500
153,Chinstrap,Dream,50.0,19.5,196.0,3900.0,Male,3.900
154,Chinstrap,Dream,51.3,19.2,193.0,3650.0,Male,3.650
155,Chinstrap,Dream,45.4,18.7,188.0,3525.0,Female,3.525
156,Chinstrap,Dream,52.7,19.8,197.0,3725.0,Male,3.725


### Transform

Transforms return a transformed version of the data that has the same shape as the input. This is useful when creating new columns that depend on some grouped data, for instance creating group-wise means. Here's an example using the datetime group to subtract a yearly mean. First let's create some synthetic data with some data, a datetime index, and some groups:

In [37]:
index = pd.date_range("1/1/2000", periods=10, freq="Q")
data = np.random.randint(0, 10, (10, 2))
df = pd.DataFrame(data, index=index, columns=["values1", "values2"])
df["type"] = np.random.choice(["group" + str(i) for i in range(3)], 10)
df

,values1,values2,type
2000-03-31,8,9,group0
2000-06-30,9,5,group1
2000-09-30,0,2,group1
2000-12-31,7,3,group0
2001-03-31,0,4,group1
2001-06-30,2,0,group2
2001-09-30,3,3,group0
2001-12-31,1,2,group2
2002-03-31,5,9,group2
2002-06-30,0,1,group1


Now we take the yearly means by type. `pd.Grouper(freq='A')` is an instruction to take the `A`nnual mean using the given datetime index. You can group on as many coloumns and/or index properties as you like: this example groups by a property of the datetime index and on the `type` column, but performs the computation on the `values1` column.

In [38]:
df["v1_demean_yr_type"] = df.groupby([pd.Grouper(freq="A"), "type"])[
    "values1"
].transform(lambda x: x - x.mean())
df

,values1,values2,type,v1_demean_yr_type
2000-03-31,8,9,group0,0.5
2000-06-30,9,5,group1,4.5
2000-09-30,0,2,group1,-4.5
2000-12-31,7,3,group0,-0.5
2001-03-31,0,4,group1,0.0
2001-06-30,2,0,group2,0.5
2001-09-30,3,3,group0,0.0
2001-12-31,1,2,group2,-0.5
2002-03-31,5,9,group2,0.0
2002-06-30,0,1,group1,0.0


You'll have seen there's a `lambda` keyword here. Lambda (or anonymous) functions have a rich history in mathematics, and were used by scientists such as Church and Turing to create proofs about what is computable *before electronic computers existed*. They can be used to define compact functions:

In [39]:
multiply_plus_one = lambda x, y: x * y + 1
multiply_plus_one(3, 4)

13

### Apply


Both regular functions and lambda functions can be used with the more general apply method, which takes a function and applies it to a given axis (`axis=0` for index, `axis=1` for columns):

In [40]:
df["val1_times_val2"] = df.apply(lambda row: row["values1"] * row["values2"], axis=1)
df

,values1,values2,type,v1_demean_yr_type,val1_times_val2
2000-03-31,8,9,group0,0.5,72
2000-06-30,9,5,group1,4.5,45
2000-09-30,0,2,group1,-4.5,0
2000-12-31,7,3,group0,-0.5,21
2001-03-31,0,4,group1,0.0,0
2001-06-30,2,0,group2,0.5,0
2001-09-30,3,3,group0,0.0,9
2001-12-31,1,2,group2,-0.5,2
2002-03-31,5,9,group2,0.0,45
2002-06-30,0,1,group1,0.0,0


Of course, the much easier way to do this very common operation is `df['val1_times_val2'] = df['values1']*df['values2']`, but there are times when you need to run more complex functions element-wise and, for those, `apply` is really useful.

### Assign

Assign is a method that allows you to return a new object with all the original columns in addition to new ones. Existing columns that are re-assigned will be overwritten. This is *really* useful when you want to perform a bunch of operations together in a concise way and keep the original columns. For instance, to demean the 'values1' column by year-type and to recompute the 'val1_times_val2' column using the newly demeaned 'values1' column:

In [41]:
df.assign(
    values1=(
        df.groupby([pd.Grouper(freq="A"), "type"])["values1"].transform(
            lambda x: x - x.mean()
        )
    ),
    val1_times_val2=lambda x: x["values1"] * x["values2"],
)

,values1,values2,type,v1_demean_yr_type,val1_times_val2
2000-03-31,0.5,9,group0,0.5,4.5
2000-06-30,4.5,5,group1,4.5,22.5
2000-09-30,-4.5,2,group1,-4.5,-9.0
2000-12-31,-0.5,3,group0,-0.5,-1.5
2001-03-31,0.0,4,group1,0.0,0.0
2001-06-30,0.5,0,group2,0.5,0.0
2001-09-30,0.0,3,group0,0.0,0.0
2001-12-31,-0.5,2,group2,-0.5,-1.0
2002-03-31,0.0,9,group2,0.0,0.0
2002-06-30,0.0,1,group1,0.0,0.0


## Time series, resampling, and rolling windows

The support for time series and the datetime type is excellent in pandas. It is very easy to manipulate datetimes. The [relevant part](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html) of the documentation has more info; here we'll just see a couple of the most important bits. First, let's create some synthetic data to work with:

In [42]:
def recursive_ts(n, x=0.05, beta=0.6, alpha=0.2):
    shock = np.random.normal(loc=0, scale=0.6)
    if n == 0:
        return beta * x + alpha + shock
    else:
        return beta * recursive_ts(n - 1, x=x) + alpha + shock


t_series = np.cumsum([recursive_ts(n) for n in range(12)])
index = pd.date_range("1/1/2000", periods=12, freq="M")
df = pd.DataFrame(t_series, index=index, columns=["values"])
df.loc["2000-08-31", "values"] = np.nan
df

,values
2000-01-31,0.418852
2000-02-29,1.694447
2000-03-31,1.920990
2000-04-30,3.752046
2000-05-31,3.467453
2000-06-30,3.167989
2000-07-31,3.959431
2000-08-31,NaN
2000-09-30,4.884800
2000-10-31,4.374416


Now let's imagine that there are a number of issues with this time series. First, it's been recorded wrong: it actually refers to the start of the next month, not the end of the previous as recorded; second, there's a missing number we want to interpolate; third, we want to take the difference of it to get to something stationary; fourth, we'd like to add a lagged column. We can do all of those things!


In [43]:
# Change freq to next month start
df.index += pd.tseries.offsets.DateOffset(days=1)

df["values"] = df["values"].interpolate(method="time")
df["diff_values"] = df["values"].diff(1)
df["lag_diff_values"] = df["diff_values"].shift(1)
df

,values,diff_values,lag_diff_values
2000-02-01,0.418852,NaN,NaN
2000-03-01,1.694447,1.275595,NaN
2000-04-01,1.920990,0.226543,1.275595
2000-05-01,3.752046,1.831056,0.226543
2000-06-01,3.467453,-0.284593,1.831056
2000-07-01,3.167989,-0.299464,-0.284593
2000-08-01,3.959431,0.791442,-0.299464
2000-09-01,4.429701,0.470269,0.791442
2000-10-01,4.884800,0.455099,0.470269
2000-11-01,4.374416,-0.510384,0.455099


Two other useful time series functions to be aware of are `resample` and `rolling`. `resample` can upsample or downsample time series. Downsampling is by aggregation, eg `df['values].resample('Q').mean()` to downsample to quarterly ('Q') frequency by taking the mean within each quarter. Upsampling involves a choice about how to fill in the missing values; examples of options are `bfill` (backfill) and `ffill` (forwards fill).

Rolling is for taking rolling aggregations, as you'd expect; for example, the 3-month rolling mean of our first difference time series:

In [44]:
df["diff_values"].rolling(3).mean()

2000-02-01         NaN
2000-03-01         NaN
2000-04-01         NaN
2000-05-01    1.111065
2000-06-01    0.591002
2000-07-01    0.415666
2000-08-01    0.069128
2000-09-01    0.320749
2000-10-01    0.572270
2000-11-01    0.138328
2000-12-01    0.145057
2001-01-01   -0.058610
Name: diff_values, dtype: float64


## Method chaining

Sometimes, rather than splitting operations out into multiple lines, it can be more concise and clear to chain methods together. A typical time you might do this is when reading in a dataset and perfoming all of the initial cleaning. Tom Augsperger has a [great tutorial](https://tomaugspurger.github.io/method-chaining) on this, which I've reproduced parts of here. For more info on the `pipe` function used below, check out these short [video tutorials](https://calmcode.io/pandas-pipe/introduction.html).

To chain methods together, both the input and output must be a pandas dataframe. Many functions already do input and output these, for example the `df.rename(columns={'old_col': 'new_col'})` takes in `df` and outputs a dataframe with one column name changed.

But occassionally, we'll want to use a function that we've defined (rather than an already existing one). For that, we need the `pipe` method; it 'pipes' the result of one operation to the next operation. When objects are being passed through multiple functions, this can be much clearer. Compare, for example,

```python
f(g(h(df), g_arg=a), f_arg=b)
```

that is, dataframe `df` is being passed to function `h`, and the results of that are being passed to a function `g` that needs a key word argument `g_arg`, and the results of *that* are being passed to a function `f` that needs keyword argument `f_arg`. The nested structure is barely readable. Compare this with

```python
(df.pipe(h)
   .pipe(g, g_arg=a)
   .pipe(f, f_arg=b)
)  
```

Let's see a method chain in action on a real dataset so you get a feel for it. We'll use 1,000 rows of flight data from BTS (a popular online dataset for demos of data cleaning!). TODO use github path. (For further info on method chaining in Python, [see these videos](https://calmcode.io/method-chains/introduction.html)--but be aware they assume advanced knowledge of the language.)

In [45]:
df = pd.read_csv(
    "https://github.com/aeturrell/coding-for-economists/raw/main/data/flights1kBTS.csv",
    index_col=0,
)
df.head()

,FL_DATE,OP_UNIQUE_CARRIER,TAIL_NUM,ORIGIN,ORIGIN_CITY_NAME,DEST,DEST_CITY_NAME,DEP_TIME,ARR_TIME,ARR_DELAY,CANCELLATION_CODE,AIR_TIME,DISTANCE,Unnamed: 13
403296,2020-01-20,G4,231NV,SFB,"Sanford, FL",AVL,"Asheville, NC",2011.0,2137.0,6.0,NaN,64.0,465.0,NaN
126199,2020-01-06,MQ,N687JS,ORD,"Chicago, IL",MSN,"Madison, WI",1837.0,1927.0,-16.0,NaN,26.0,109.0,NaN
417268,2020-01-20,YV,N516LR,IAD,"Washington, DC",DTW,"Detroit, MI",1738.0,1915.0,14.0,NaN,60.0,383.0,NaN
67589,2020-01-04,OO,N804SK,SAV,"Savannah, GA",DTW,"Detroit, MI",653.0,844.0,-26.0,NaN,91.0,705.0,NaN
102760,2020-01-05,WN,N789SW,SAN,"San Diego, CA",TUS,"Tucson, AZ",732.0,941.0,-19.0,NaN,53.0,368.0,NaN


We'll try and do a number of operations in one go: putting column titles in lower case, discarding useless columns, creating precise depature and arrival times, turning some of the variables into categoricals, creating a demeaned delay time, and creating a new categorical column for distances according to quantiles that will be called 'near', 'less near', 'far', and 'furthest'. Some of these operations require a separate function, so we first define those. When we do the cleaning, we'll pipe our dataframe to those functions (optionally passing any arguments).

In [46]:
def extract_city_name(df):
    """
    Chicago, IL -> Chicago for origin_city_name and dest_city_name
    """
    cols = ["origin_city_name", "dest_city_name"]
    city = df[cols].apply(lambda x: x.str.extract("(.*), \w{2}", expand=False))
    df = df.copy()
    df[["origin_city_name", "dest_city_name"]] = city
    return df


def time_to_datetime(df, columns):
    """
    Combine all time items into datetimes.

    2014-01-01,0914 -> 2014-01-01 09:14:00
    """
    df = df.copy()

    def converter(col):
        timepart = (
            col.astype(str)
            .str.replace("\.0$", "")  # NaNs force float dtype
            .str.pad(4, fillchar="0")
        )
        return pd.to_datetime(
            df["fl_date"]
            + " "
            + timepart.str.slice(0, 2)
            + ":"
            + timepart.str.slice(2, 4),
            errors="coerce",
        )

    df[columns] = df[columns].apply(converter)
    return df


df = (
    df.drop([x for x in df.columns if "Unnamed" in x], axis=1)
    .rename(columns=str.lower)
    .pipe(extract_city_name)
    .pipe(time_to_datetime, ["dep_time", "arr_time"])
    .assign(
        fl_date=lambda x: pd.to_datetime(x["fl_date"]),
        dest=lambda x: pd.Categorical(x["dest"]),
        origin=lambda x: pd.Categorical(x["origin"]),
        tail_num=lambda x: pd.Categorical(x["tail_num"]),
        arr_delay=lambda x: pd.to_numeric(x["arr_delay"]),
        op_unique_carrier=lambda x: pd.Categorical(x["op_unique_carrier"]),
        arr_delay_demean=lambda x: x["arr_delay"] - x["arr_delay"].mean(),
        distance_group=lambda x: (
            pd.qcut(x["distance"], 4, labels=["near", "less near", "far", "furthest"])
        ),
    )
)
df.head()

/var/folders/x6/ffnr59f116l96_y0q0bjfz7c0000gn/T/ipykernel_5032/1424200124.py:22: FutureWarning: The default value of regex will change from True to False in a future version.
  col.astype(str)


,fl_date,op_unique_carrier,tail_num,origin,origin_city_name,dest,dest_city_name,dep_time,arr_time,arr_delay,cancellation_code,air_time,distance,arr_delay_demean,distance_group
403296,2020-01-20,G4,231NV,SFB,Sanford,AVL,Asheville,2020-01-20 20:11:00,2020-01-20 21:37:00,6.0,NaN,64.0,465.0,8.544995,less near
126199,2020-01-06,MQ,N687JS,ORD,Chicago,MSN,Madison,2020-01-06 18:37:00,2020-01-06 19:27:00,-16.0,NaN,26.0,109.0,-13.455005,near
417268,2020-01-20,YV,N516LR,IAD,Washington,DTW,Detroit,2020-01-20 17:38:00,2020-01-20 19:15:00,14.0,NaN,60.0,383.0,16.544995,less near
67589,2020-01-04,OO,N804SK,SAV,Savannah,DTW,Detroit,2020-01-04 06:53:00,2020-01-04 08:44:00,-26.0,NaN,91.0,705.0,-23.455005,far
102760,2020-01-05,WN,N789SW,SAN,San Diego,TUS,Tucson,2020-01-05 07:32:00,2020-01-05 09:41:00,-19.0,NaN,53.0,368.0,-16.455005,less near


### Pyjanitor and more extensive method chaining

Although there's enough support for functional style method chaining to get by, you might find that doing this or that operation isn't always as efficient or as concise as you'd like. Let's look at two examples where this bites and what a wonderful little extension to **pandas** called [**Pyjanitor**](https://pyjanitor.readthedocs.io) does about it.

First, you'll have seen that the syntax for assigning a new column as part of a method chain is relatively fussy, using as it does the `new_column=lambda x: func(x['old_column])` syntax. If you already have a dataframe named `df`, **pyjanitor** gives you the option to create new columns like so (using an example from the documentation):


```python

df = (
    df.add_columns(
        prop_late_departures=df.num_departing_late / df.total_num_trips,
        prop_late_arrivals=df.num_arriving_late / df.total_num_trips
    )
)
```

What's great about this is not only that it's cleaner to read, but that all you need do to get the functionality is to import the pyjanitor library and the extra functions will appear magically as options to use on the usual pandas dataframes.

Pyjanitor isn't just about method chaining--it does a lot of other things too, like introducing a bunch of convenience commands with easily understandable verbs behind them. (Though do note it has a hefty download size.)

## Reshaping data

The main options for reshaping data are `pivot`, `melt`, `stack`, `unstack`, `pivot_table`, `get_dummies`, `cross_tab`, and `explode`. We’ll look at some of these here.



### Pivoting data from tidy to, err, untidy

At the start of this chapter, I said you should use tidy data--one row per observation, one column per variable--whenever you can. But there are times when you will want to take your lovingly prepared tidy data and pivot it into a wider format. `pivot` and `pivot_table` help you to do that.

![](https://pandas.pydata.org/docs/_images/reshaping_pivot.png)

This can be especially useful for time series data, where operations like `shift` or `diff` are typically applied assuming that an entry in one row follows (in time) from the one above. Here's an example:

In [47]:
data = {
    "value": np.random.randn(20),
    "variable": ["A"] * 10 + ["B"] * 10,
    "date": (
        list(pd.date_range("1/1/2000", periods=10, freq="M"))
        + list(pd.date_range("1/1/2000", periods=10, freq="M"))
    ),
}
df = pd.DataFrame(data, columns=["date", "variable", "value"])
df.sample(5)

,date,variable,value
14,2000-05-31,B,-0.333101
16,2000-07-31,B,0.274173
12,2000-03-31,B,-2.295103
17,2000-08-31,B,-0.514910
13,2000-04-30,B,-1.195929


If we just run `shift` on this, it's going to shift variable B's and A's together. So we pivot to a wider format (and then we can shift safely).

In [48]:
df.pivot(index="date", columns="variable", values="value").shift(1)

variable,A,B
date,,
2000-01-31,NaN,NaN
2000-02-29,1.344549,0.460888
2000-03-31,0.493515,0.662374
2000-04-30,-0.237004,-2.295103
2000-05-31,0.057285,-1.195929
2000-06-30,-0.707071,-0.333101
2000-07-31,0.546665,-0.791391
2000-08-31,0.942500,0.274173
2000-09-30,-2.979597,-0.514910



### Melt

`melt` can help you go from untidy to tidy data (from wide data to long data), and is a *really* good one to remember. Of course, I have to look at the documentation every single time myself, but I'm sure you'll do better.

![](https://pandas.pydata.org/docs/_images/reshaping_melt.png)

Here's an example of it in action:

In [49]:
df = pd.DataFrame(
    {
        "first": ["John", "Mary"],
        "last": ["Doe", "Bo"],
        "height": [5.5, 6.0],
        "weight": [130, 150],
    }
)
print("\n Unmelted: ")
print(df)
print("\n Melted: ")
df.melt(id_vars=["first", "last"], var_name="quantity")


 Unmelted: 
  first last  height  weight
0  John  Doe     5.5     130
1  Mary   Bo     6.0     150

 Melted: 


,first,last,quantity,value
0,John,Doe,height,5.5
1,Mary,Bo,height,6.0
2,John,Doe,weight,130.0
3,Mary,Bo,weight,150.0


If you don't wan the headscratching of melt, there's also `wide_to_long`, which is really useful for typical data cleaning cases where you have data like this:

In [50]:
df = pd.DataFrame(
    {
        "A1970": {0: "a", 1: "b", 2: "c"},
        "A1980": {0: "d", 1: "e", 2: "f"},
        "B1970": {0: 2.5, 1: 1.2, 2: 0.7},
        "B1980": {0: 3.2, 1: 1.3, 2: 0.1},
        "X": dict(zip(range(3), np.random.randn(3))),
        "id": dict(zip(range(3), range(3))),
    }
)
df

,A1970,A1980,B1970,B1980,X,id
0,a,d,2.5,3.2,-1.598324,0
1,b,e,1.2,1.3,1.261497,1
2,c,f,0.7,0.1,-0.580701,2


i.e. data where there are different variables and time periods across the columns. Wide to long is going to let us give info on what the stubnames are ('A', 'B'), the name of the variable that's always across columns (here, a year), any values (X here), and an id column.

In [51]:
pd.wide_to_long(df, ["A", "B"], i="id", j="year")

,,X,A,B
id,year,,,
0,1970,-1.598324,a,2.5
1,1970,1.261497,b,1.2
2,1970,-0.580701,c,0.7
0,1980,-1.598324,d,3.2
1,1980,1.261497,e,1.3
2,1980,-0.580701,f,0.1


### Stack and unstack

Stack, `stack()` is a shortcut for taking a single type of wide data variable from columns and turning it into a long form dataset, but with an extra index.

![](https://pandas.pydata.org/docs/_images/reshaping_stack.png)

Unstack, `unstack()` unsurprisingly does the same operation, but in reverse.

![](https://pandas.pydata.org/docs/_images/reshaping_unstack.png)

Let's define a multi-index dataframe to demonstrate this:

In [52]:
tuples = list(
    zip(
        *[
            ["bar", "bar", "baz", "baz", "foo", "foo", "qux", "qux"],
            ["one", "two", "one", "two", "one", "two", "one", "two"],
        ]
    )
)
index = pd.MultiIndex.from_tuples(tuples, names=["first", "second"])
df = pd.DataFrame(np.random.randn(8, 2), index=index, columns=["A", "B"])
df

A         B
first second                    
bar   one    -0.069768 -0.224551
      two    -1.626197 -0.071688
baz   one     0.086031 -1.369931
      two    -1.010371 -0.943652
foo   one     0.920108 -1.268939
      two    -1.342901  0.303793
qux   one    -1.682761 -0.856373
      two    -1.075533 -1.665213

Let's stack this to create a tidy dataset:

In [53]:
df = df.stack()
df

first  second   
bar    one     A   -0.069768
               B   -0.224551
       two     A   -1.626197
               B   -0.071688
baz    one     A    0.086031
               B   -1.369931
       two     A   -1.010371
               B   -0.943652
foo    one     A    0.920108
               B   -1.268939
       two     A   -1.342901
               B    0.303793
qux    one     A   -1.682761
               B   -0.856373
       two     A   -1.075533
               B   -1.665213
dtype: float64

Now let's see unstack but, instead of unstacking the 'A', 'B' variables we began with, let's unstack the 'first' column by passing `level=0` (the default is to unstack the innermost index). This diagram shows what's going on:

![](https://pandas.pydata.org/docs/_images/reshaping_unstack_0.png)

And here's the code:

In [54]:
df.unstack(level=0)

first          bar       baz       foo       qux
second                                          
one    A -0.069768  0.086031  0.920108 -1.682761
       B -0.224551 -1.369931 -1.268939 -0.856373
two    A -1.626197 -1.010371 -1.342901 -1.075533
       B -0.071688 -0.943652  0.303793 -1.665213

### Get dummies

This is a really useful reshape command for when you want (explicit) dummies in your dataframe. When running simple regressions, you can achieve the same effect by declaring the column only be included as a fixed effect, but there are some machine learning packages where converting to dummies may be easier.

Here's an example:

In [55]:
df = pd.DataFrame(
    {"group_var": ["group1", "group2", "group3"], "B": ["c", "c", "b"], "C": [1, 2, 3]}
)
print(df)

pd.get_dummies(df, columns=["group_var"])

  group_var  B  C
0    group1  c  1
1    group2  c  2
2    group3  b  3


,B,C,group_var_group1,group_var_group2,group_var_group3
0,c,1,1,0,0
1,c,2,0,1,0
2,b,3,0,0,1



## Combining data

**pandas** has a really rich set of options for combining one or more dataframes. The two most important are concatenate and merge.

### Concatenate

If you have two or more dataframes with the same index or the same columns, you can glue them together into a single dataframe using `pd.concat`. 

![](https://pandas.pydata.org/docs/_images/08_concat_row.svg)

For the same columns, pass `axis=0` to glue the index together; for the same index, pass `axis=1` to glue the columns together. The concatenate function will typically be used on a list of dataframes.

If you want to track where the original data came from in the final dataframe, use the `keys` keyword.

Here's an example using data on two different states' populations that also makes uses of the `keys` option:

In [56]:
base_url = "http://www.stata-press.com/data/r14/"
state_codes = ["ca", "il"]
end_url = "pop.dta"

# This grabs the two dataframes, one for each state
list_of_state_dfs = [pd.read_stata(base_url + state + end_url) for state in state_codes]

# Concatenate the list of dataframes
df = pd.concat(list_of_state_dfs, keys=state_codes, axis=0)
df

county      pop
ca 0  Los Angeles  9878554
   1       Orange  2997033
   2      Ventura   798364
il 0         Cook  5285107
   1       DeKalb   103729
   2         Will   673586

### Merge

There are so many options for merging dataframes using `pd.merge(left, right, on=..., how=...` that we won't be able to cover them all here. The most important features are: the two dataframes to be merged, what variables (aka keys) to merge on (and these can be indexes) via `on=`, and *how* to do the merge (eg left, right, outer, inner) via `how=`. This diagram shows an example of a merge using keys from the left-hand dataframe:

![](https://pandas.pydata.org/docs/_images/08_merge_left.svg)

The `how=` keyword works in the following ways:
- `how='left'` uses keys from the left dataframe only to merge.
- `how='right'` uses keys from the right dataframe only to merge.
- `how='inner'` uses keys that appear in both dataframes to merge.
- `how='outer'` uses the cartesian product of keys in both dataframes to merge on.

Let's see examples of some of these:

In [57]:
left = pd.DataFrame(
    {
        "key1": ["K0", "K0", "K1", "K2"],
        "key2": ["K0", "K1", "K0", "K1"],
        "A": ["A0", "A1", "A2", "A3"],
        "B": ["B0", "B1", "B2", "B3"],
    }
)
right = pd.DataFrame(
    {
        "key1": ["K0", "K1", "K1", "K2"],
        "key2": ["K0", "K0", "K0", "K0"],
        "C": ["C0", "C1", "C2", "C3"],
        "D": ["D0", "D1", "D2", "D3"],
    }
)
# Right merge
pd.merge(left, right, on=["key1", "key2"], how="right")

,key1,key2,A,B,C,D
0,K0,K0,A0,B0,C0,D0
1,K1,K0,A2,B2,C1,D1
2,K1,K0,A2,B2,C2,D2
3,K2,K0,NaN,NaN,C3,D3


Note that the key combination of K2 and K0 did not exist in the left-hand dataframe, and so its entries in the final dataframe are NaNs. But it *does* have entries because we chose the keys from the right-hand dataframe.

What about an inner merge?

In [58]:
pd.merge(left, right, on=["key1", "key2"], how="inner")

,key1,key2,A,B,C,D
0,K0,K0,A0,B0,C0,D0
1,K1,K0,A2,B2,C1,D1
2,K1,K0,A2,B2,C2,D2


Now we see that the combination K2 and K0 are excluded because they didn't exist in the overlap of keys in both dataframes.

Finally, let's take a look at an outer merge that comes with some extra info via the `indicator` keyword:

In [59]:
pd.merge(left, right, on=["key1", "key2"], how="outer", indicator=True)

,key1,key2,A,B,C,D,_merge
0,K0,K0,A0,B0,C0,D0,both
1,K0,K1,A1,B1,NaN,NaN,left_only
2,K1,K0,A2,B2,C1,D1,both
3,K1,K0,A2,B2,C2,D2,both
4,K2,K1,A3,B3,NaN,NaN,left_only
5,K2,K0,NaN,NaN,C3,D3,right_only


Now we can see that the products of all key combinations are here. The `indicator=True` option has caused an extra column to be added, called '_merge', that tells us which dataframe the keys on that row came from.

For more on the options, see the **pandas** [merging documentation](https://pandas.pydata.org/docs/user_guide/merging.html#database-style-dataframe-or-named-series-joining-merging).

## (Advanced) alternatives to **pandas**

Feel free to skip this section if you're just interested in getting going as quickly as possible.

**pandas** isn't the only game in town, not by a long way—though it's by far the best supported and the most fully featured. But it's always good to have options—or, put another way, options have value! Other dataframe libraries may have a syntax that you prefer or provide a speed-up (perhaps in certain situations, for example when working with very large datasets).

If you're specifically interested in how different dataframe options perform on increasingly large datasets, take a look at the benchmarks [here](https://h2oai.github.io/db-benchmark/).

If you need speed, the Python and Rust library Polars is currently the fastest performer on small and big data in any of Python, Julia, R, and Rust.

Here's a quick run-through of some alternatives to **pandas**:

### Datatable

[**datatable**](https://datatable.readthedocs.io/en/latest/) is another dataframe based way to do analysis, and it has quite a different syntax to **pandas** for data manipulation. **datatable** is very fast, not the fastest dataframe option out there, but it holds its own. Its other major advantage is that it is comfortable running on extremely large (think 50GB) datasets. Most other dataframe packages (including in other languages) cannot cope with this.

I am indebted to the very well-written datatable documentation for the rest of this demonstration.

In datatable, almost all operations are achieved via so-called 'square-bracket notation'. Operations with a (data) Frame are almost all expressed in the form

```python
DT[i, j, ...]
```

where `DT` is a datatable dataframe, `i` is the row selector, `j` is the column selector, and `...` is a space that could be filled by other commands (potentially several). However, `i` and `j` aren't just positional, as they might be in a **numpy** array or a list, they do a lot more than that. 

Datatable allows `i` to be anything that can conceivably be interpreted as a row selector: an integer to select a row, a slice, a range, a list of integers, a list of slices, an expression, a boolean-valued Frame, an integer-valued Frame, an integer **numpy** array, a generator, and more.

Likewise, `j` is also really versatile. It can be used to select a column by name or position, but it will also accept a list of columns, a slice, a list of booleans indicating which columns to pick, an expression, a list of expressions, and a dictionary of expressions. The j expression can even be a python type (such as int or dt.float32), selecting all columns matching that type!

To change an entry (entries), you can use `DT[i, j] = new_value`, and to remove it (or them), it's `del DT[i, j]`.

It's probably going to be easiest to see some examples, so let's do a quick run through of some functions with the storms dataset.

In [60]:
from datatable import dt, f, by, g, join, sort, update, ifelse

DT = dt.fread("https://vincentarelbundock.github.io/Rdatasets/csv/dplyr/storms.csv")

DT.head(5)

,C0,name,year,month,day,hour,lat,long,status,category,wind,pressure,ts_diameter,hu_diameter
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,1,Amy,1975,6,27,0,27.5,−79,tropical depression,−1,25,1013,NA,NA
1,2,Amy,1975,6,27,6,28.5,−79,tropical depression,−1,25,1013,NA,NA
2,3,Amy,1975,6,27,12,29.5,−79,tropical depression,−1,25,1013,NA,NA
3,4,Amy,1975,6,27,18,30.5,−79,tropical depression,−1,25,1013,NA,NA
4,5,Amy,1975,6,28,0,31.5,−78.8,tropical depression,−1,25,1012,NA,NA


Yes, that's right, you're seeing columns colour coded by data type! Otherwise, what you can see here is that the syntax so far looks quite similar but don't worry because it's about to get weird.

The square brackets accept expressions, i.e. functions. This is achieved through a special import, simply denoted `f`, that says do this thing to the current datatable. For example, to filter by a certain value in the 'status' column (to avoid overloading with data, I'll just use head to show the first few rows):

In [61]:
DT[f.status == "tropical storm", :].head(4)

,C0,name,year,month,day,hour,lat,long,status,category,wind,pressure,ts_diameter,hu_diameter
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,9,Amy,1975,6,29,0,34.4,−75.8,tropical storm,0,35,1004,NA,NA
1,10,Amy,1975,6,29,6,34,−74.8,tropical storm,0,40,1002,NA,NA
2,11,Amy,1975,6,29,12,33.8,−73.8,tropical storm,0,45,1000,NA,NA
3,12,Amy,1975,6,29,18,33.8,−72.8,tropical storm,0,50,998,NA,NA


Let's get rid of some columns we're not using right now

In [62]:
del DT[:, "year":"long"]
DT.head(4)

,C0,name,status,category,wind,pressure,ts_diameter,hu_diameter
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,1,Amy,tropical depression,−1,25,1013,NA,NA
1,2,Amy,tropical depression,−1,25,1013,NA,NA
2,3,Amy,tropical depression,−1,25,1013,NA,NA
3,4,Amy,tropical depression,−1,25,1013,NA,NA


Now, to select only those columns that are strings:


In [63]:
DT[:, str].head(4)

,name,status
,▪▪▪▪,▪▪▪▪
0,Amy,tropical depression
1,Amy,tropical depression
2,Amy,tropical depression
3,Amy,tropical depression


Or to select only those columns that contain 'diameter'

In [64]:
DT[:, [x for x in DT.names if "diameter" in x]].tail(4)

,ts_diameter,hu_diameter
,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,161.109,23.0156
1,207.14,23.0156
2,345.234,34.5234
3,379.757,46.0312


To create a new column based on an existing one and just look at the last few columns:

In [65]:
DT[:, update(atmospheres=f.pressure / 1013.0)]
DT[:, -4:].tail(4)

,pressure,ts_diameter,hu_diameter,atmospheres
,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,990,161.109,23.0156,0.977295
1,985,207.14,23.0156,0.972359
2,980,345.234,34.5234,0.967423
3,980,379.757,46.0312,0.967423


To sort all values by multiple columns:

In [66]:
DT[:, :, sort("wind", "pressure", reverse=[True, True])].head(4)

,C0,name,status,category,wind,pressure,ts_diameter,hu_diameter,atmospheres
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,1844,Gilbert,hurricane,5,160,888,NA,NA,0.876604
1,7266,Wilma,hurricane,5,160,882,304.957,74.8007,0.870681
2,4802,Mitch,hurricane,5,155,910,NA,NA,0.898322
3,4801,Mitch,hurricane,5,155,905,NA,NA,0.893386


And to perform group-by operations:

In [67]:
DT[:, dt.mean(f.wind), by("status")]

,status,wind
,▪▪▪▪,▪▪▪▪▪▪▪▪
0,hurricane,85.9689
1,tropical depression,27.2692
2,tropical storm,45.8059


And, finally, one that I always find of great practical use--the within-group transform, with the new column going back into the original data(frame/table). In **pandas**, this is achieved by `transform`, here we use `extend`. Let's demonstrate by showing the maximum wind speed by status group:

In [68]:
DT[:, f[:].extend({"max_wind": dt.min(f.wind)}), by("status")].head(4)

,status,C0,name,category,wind,pressure,ts_diameter,hu_diameter,atmospheres,max_wind
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪
0,hurricane,53,Caroline,1,65,990,NA,NA,0.977295,65
1,hurricane,54,Caroline,1,65,990,NA,NA,0.977295,65
2,hurricane,55,Caroline,1,65,989,NA,NA,0.976308,65
3,hurricane,56,Caroline,1,70,987,NA,NA,0.974334,65


### cuDF

[**cuDF**](https://github.com/rapidsai/cudf) is still under development and so doesn't yet have all the features of **pandas**. It's a Python GPU DataFrame library built on the blisteringly fast Apache Arrow columnar memory format. The parts of the library that have been implemented follow the same commands and structure as **pandas**, so it should be easy to use it should you need to.

**cuDF** is one of the fastest dataframe library out there in any of Python, Julia, Rust, or R with the caveats that:
- this only applies to in-memory datasets, ie datasets smaller than a few GB
- it doesn't yet do everything
- it's only currently available on the Linux operating system
- you need a GPU (and associated software) to use it!

### Polars

[**Polars**](https://www.pola.rs/) is consistently the fastest dataframe library in any of Python, Julia, Rust, or R. It also uses Apache Arrow as backend. It currently consists of an 'eager' (for datasets smaller than approximately a few GB) interface that's very similar to **pandas** and a 'lazy' interface (don't worry if you don't know what that means, it's a big data thing) that is somewhat similar to spark (a big data tool). **Polars** is built on the Rust language. It's particularly effective at merging datasets.

### Plydata

[Plydata](https://plydata.readthedocs.io) uses the syntax of the **dplyr** package in the R statistical language (the package is a really important part of the data ecosystem in R). It's built around method chaining and everything being in a tidy format, which has pros and cons. Largely, the benefits are in readability and a smaller number of commands to cover the ground that you need to. The downsides are that it plays best with tidy data and when chained methods go wrong it can be more difficult to find out what the issue is.

### Datar

Another library that gets close to **dplylr** syntax is [**datar**](https://github.com/pwwang/datar). Like **plydata**, it integrates with **plotnine** for visualisation.

### Others

- [dfplyr](https://github.com/kieferk/dfply) also follows the syntax of the **dplyr** package in R but it does not appear to be under active development.

- [siuba](https://github.com/machow/siuba) is yet another **dplyr** inspired package! It is under active development. 

## Review

If you know:

- ✅ what tidy data are;
- ✅ how to create series and dataframes with different datatypes;
- ✅ how to access values in series and dataframes;
- ✅ how to perform operations on columns;
- ✅ how to chain methods;
- ✅ how to reshape data;
- ✅ how to combine different dataframes; and
- ✅ what some alternatives to **pandas** are

then you are well on your way to becoming a data analysis master!
